In [1]:
import graphlab

In [8]:
matchups = graphlab.SFrame.read_csv("2016_yahoo_matchups_week1-7.csv",delimiter='|')
weeks_elapsed = 7

Finished parsing file /Users/lukechu/Desktop/Pycharm/YahooFF2/2016_yahoo_matchups_week1-7.csv

Parsing completed. Parsed 35 lines in 0.039366 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,str,float,float,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/lukechu/Desktop/Pycharm/YahooFF2/2016_yahoo_matchups_week1-7.csv

Parsing completed. Parsed 35 lines in 0.018169 secs.

In [3]:
matchups.head()

Week,TeamOne,ProjectedScoreOne,ScoreOne,Winner,ScoreTwo,ProjectedScoreTwo,TeamTwo
1,The Golden Yogis,107.69,102.2,<,149.46,120.48,Clippernation
1,In TyGod we Trust,109.61,90.46,<,108.44,109.49,Team 8
1,dicksoutforharambe,107.68,96.92,<,112.56,114.6,NomoneyforRedzone
1,Penguins,115.82,106.22,<,134.9,114.46,Romosexual
1,Sit4Anthems,106.49,73.1,<,118.16,98.82,theoutoftowner
2,The Golden Yogis,106.49,84.2,<,110.16,112.87,In TyGod we Trust
2,Clippernation,130.55,111.52,<,123.86,99.84,theoutoftowner
2,dicksoutforharambe,103.34,104.36,>,101.46,110.47,Team 8
2,Penguins,117.62,117.62,>,102.52,111.9,NomoneyforRedzone
2,Sit4Anthems,104.85,117.76,>,92.92,105.34,Romosexual


In [4]:
# sort by score per week
matchup_week_score = {}

for i in range(1, weeks_elapsed+1): 
    matchup_week_score[i] = matchups[matchups["Week"]==i][["TeamOne", "ProjectedScoreOne", "ScoreOne" ]] # alternatively matchups[["Week"], ...].filter_by(1, "Week")
    matchup_week_score[i].rename({"TeamOne":"Team", "ProjectedScoreOne":"ProjectedScore", "ScoreOne":"Score"})
    
    # collision possible, but there are only 16 weeks
    matchup_week_score[i + 20] = matchups[matchups["Week"]==i][["TeamTwo", "ProjectedScoreTwo", "ScoreTwo" ]] # alternatively matchups[["Week"], ...].filter_by(1, "Week")
    matchup_week_score[i + 20].rename({"TeamTwo":"Team", "ProjectedScoreTwo":"ProjectedScore", "ScoreTwo":"Score"})

    matchup_week_score[i] = matchup_week_score[i].append(matchup_week_score[i+20])
    # remember that a column is an SArray (and a row is a dict)
    # alternatively (would be longer) could use .add_row_number, .rename, and reorder column to be last column
    matchup_week_score[i] = matchup_week_score[i].sort(["Score"]).add_column(graphlab.SArray(range(0,matchup_week_score[i].num_rows())), name ="NumberBeaten")

matchup_week_score

{1: Columns:
 	Team	str
 	ProjectedScore	float
 	Score	float
 	NumberBeaten	int
 
 Rows: 10
 
 Data:
 +--------------------+----------------+--------+--------------+
 |        Team        | ProjectedScore | Score  | NumberBeaten |
 +--------------------+----------------+--------+--------------+
 |    Sit4Anthems     |     106.49     |  73.1  |      0       |
 | In TyGod we Trust  |     109.61     | 90.46  |      1       |
 | dicksoutforharambe |     107.68     | 96.92  |      2       |
 |  The Golden Yogis  |     107.69     | 102.2  |      3       |
 |      Penguins      |     115.82     | 106.22 |      4       |
 |       Team 8       |     109.49     | 108.44 |      5       |
 | NomoneyforRedzone  |     114.6      | 112.56 |      6       |
 |   theoutoftowner   |     98.82      | 118.16 |      7       |
 |     Romosexual     |     114.46     | 134.9  |      8       |
 |   Clippernation    |     120.48     | 149.46 |      9       |
 +--------------------+----------------+--------+-----

In [9]:
# Calculate Expected Wins. EW = sum( # of opponents beaten each week ) / # of opponents

# For each team, group by number beaten
#simple way to initalize new SFrame with same structure and skip an append
matchup_scores = matchup_week_score[1]
for i in range(2, weeks_elapsed + 1):
    matchup_scores = matchup_scores.append(matchup_week_score[i])

# group by sum of total Beaten    
teams_beaten = matchup_scores.groupby(key_columns = "Team", operations = {"TotalBeaten": graphlab.aggregate.SUM("NumberBeaten")})
teams_beaten.add_column(teams_beaten["TotalBeaten"]/ 9, "ExpectedWins") # 9 opponents
teams_beaten["TotalBeaten"] = teams_beaten["TotalBeaten"]/ weeks_elapsed # 
teams_beaten.rename({"TotalBeaten": "AvgBeatPerWeek"})
teams_beaten.sort("ExpectedWins", ascending = False)

# SFrames don't have round function?

Team,AvgBeatPerWeek,ExpectedWins
Penguins,6.28571428571,4.88888888889
Romosexual,6.14285714286,4.77777777778
Clippernation,4.57142857143,3.55555555556
dicksoutforharambe,4.42857142857,3.44444444444
theoutoftowner,4.28571428571,3.33333333333
The Golden Yogis,4.14285714286,3.22222222222
Team 8,4.0,3.11111111111
In TyGod we Trust,3.71428571429,2.88888888889
Sit4Anthems,3.71428571429,2.88888888889
NomoneyforRedzone,3.71428571429,2.88888888889
